# Inference and Eval

In [2]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from torch import bfloat16
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig, PeftModel
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

## Code to load huggingface models with and without lora trained weights

In [1]:

def load_lora_model(
        llm_model_name: str,
        lora_model_weights: str,
        device: str = "auto",
        use_4bit: bool = False,
        use_8bit: bool = False
    ):

    model_config = transformers.AutoConfig.from_pretrained(
        llm_model_name,
        trust_remote_code=True,
    )

    # quantization - https://huggingface.co/blog/4bit-transformers-bitsandbytes
    bnb_config = None
    if use_4bit:
        bnb_config = transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            # # Commented to test V100
            #bnb_4bit_compute_dtype=bfloat16
        )
    
    model = transformers.AutoModelForCausalLM.from_pretrained(
        llm_model_name,
        load_in_8bit=use_8bit,
        trust_remote_code=True,
        config=model_config,
        device_map=device,
        # Only put quantize here!
        quantization_config=bnb_config
    )
    
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        llm_model_name,
        trust_remote_code=True,
    )

    # Add the Lora weights to the base model
    model = PeftModel.from_pretrained(
        model=model,
        model_id=lora_model_weights,
    )

    return model, tokenizer


def load_huggingface_base_model(
    model_name:str="microsoft/phi-2", 
    dtype:str="bfloat16", 
    use_4bit:bool=False, use_8bit:bool=False
    ):

    compute_dtype = getattr(torch, dtype)

    bnb_config=None
    if use_4bit:
        bnb_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
        )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=use_8bit,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=bnb_config, 
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                            trust_remote_code=True,
                                            )
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


def get_pipeline(model, tokenizer, max_tokens=25, temp=0.0):
    pipe = pipeline(task="text-generation", 
        model=model, 
        tokenizer=tokenizer,
        max_new_tokens=max_tokens, 
        temperature=temp,
    )    
    return pipe


## Prompt

The data consists of sentences. These are then put in prompts. 

The training set includes the correct sentiment/label; The test set does not.

In [4]:
sentence = "( ADP News ) - Finnish handling systems provider Cargotec Oyj ( HEL : CGCBV ) announced on Friday it won orders worth EUR 10 million ( USD 13.2 m ) to deliver linkspans to Jordan , Morocco and Ireland ."
print(sentence)

( ADP News ) - Finnish handling systems provider Cargotec Oyj ( HEL : CGCBV ) announced on Friday it won orders worth EUR 10 million ( USD 13.2 m ) to deliver linkspans to Jordan , Morocco and Ireland .


In [5]:
prompt = f"""The sentiment of the following phrase: '{sentence}' is 

 Positive
            
 Negative
            
 Neutral
            
 Cannot be determined

Solution: The correct option is"""

print(prompt)

The sentiment of the following phrase: '( ADP News ) - Finnish handling systems provider Cargotec Oyj ( HEL : CGCBV ) announced on Friday it won orders worth EUR 10 million ( USD 13.2 m ) to deliver linkspans to Jordan , Morocco and Ireland .' is 

 Positive
            
 Negative
            
 Neutral
            
 Cannot be determined

Solution: The correct option is


## Base Model Load and Inference

In [6]:
model_name = "microsoft/phi-2"

In [7]:
base_model, base_tokenizer = load_huggingface_base_model(model_name)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.20it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
base_pipe = get_pipeline(base_model, base_tokenizer, max_tokens=80)

In [9]:
result = base_pipe(prompt, pad_token_id=base_pipe.tokenizer.eos_token_id)
print(result[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


The sentiment of the following phrase: '( ADP News ) - Finnish handling systems provider Cargotec Oyj ( HEL : CGCBV ) announced on Friday it won orders worth EUR 10 million ( USD 13.2 m ) to deliver linkspans to Jordan , Morocco and Ireland .' is 

 Positive
            
 Negative
            
 Neutral
            
 Cannot be determined

Solution: The correct option is  C Neutral

Explanation: The sentiment of a phrase is the attitude or emotion that the speaker or writer conveys through their words. It can be positive, negative, or neutral, depending on the tone and context of the phrase. To determine the sentiment of a phrase, we can look for clues such as the choice of words, the punctuation, the modifiers, and the comparison


#### Things to note about the base model response above.
- It's wrong. The sentence is positive from a financial point of view
- It adds the C which isn't in the prompt but also the Explanation starts with 'Explanation'. These come from the original training.
- The explanation is wrong!


## Finetuned Lora Model Load and Inference

In [10]:
# The folder where the lora model weights were saved after training using code: trainer.model.save_pretrained("phi2_sentiment_model")
lora_model_fldr = "./phi2_sentiment_model"

In [11]:
lora_model, lora_tokenizer = load_lora_model(model_name, lora_model_fldr)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.20it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
ft_lora_pipe = get_pipeline(lora_model, lora_tokenizer, max_tokens=80)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [13]:
result = ft_lora_pipe(prompt, pad_token_id=ft_lora_pipe.tokenizer.eos_token_id)
print(result[0]['generated_text'])

The sentiment of the following phrase: '( ADP News ) - Finnish handling systems provider Cargotec Oyj ( HEL : CGCBV ) announced on Friday it won orders worth EUR 10 million ( USD 13.2 m ) to deliver linkspans to Jordan , Morocco and Ireland .' is 

 Positive
            
 Negative
            
 Neutral
            
 Cannot be determined

Solution: The correct option is positive. The phrase 'won orders worth EUR 10 million ( USD 13.2 m )' indicates that the company received a positive amount of money from the customers. Therefore, the sentiment of the phrase is positive.



#### Things to note about the fine-tuned response
- It's correct, there is no 'Explanation:'. The fine-tuning removed that and the response is closer to the training set prompt.
- Also the reasoning is correct and different (even though we didn't fine-tune on reasoning).

## Calling Finetuned Lora model from Langchain

In [14]:
# Langchain
langchain_pipe = HuggingFacePipeline(pipeline=ft_lora_pipe)
langchain_pipe.invoke(prompt)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


" positive. The phrase 'won orders worth EUR 10 million ( USD 13.2 m )' indicates that the company received a positive amount of money from the customers. Therefore, the sentiment of the phrase is positive.\n"

AttributeError: module 'torch.distributed' has no attribute 'is_world_process_zero'